# Imports

In [99]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.manifold import Isomap
import pickle
import time

# Dataset Analysis

In [3]:
# Load the dataset
df = pd.read_csv('csv/train_data.csv')

# Unneeded
df = df.drop(columns=['Unnamed: 0'])
df = df.drop_duplicates()

# Labeled subset (for training the supervised model)
df_labeled = df[df['SurvivalTime'].notna()]

Check for dataset metrics

In [ ]:
# Pandas profiler
profile = ProfileReport(df)
profile.to_notebook_iframe()

# Data Preprocessing

In [44]:
# TEST CELL

from statsmodels.multivariate.pca import PCA as PCA_sm

data = df.drop(columns=['SurvivalTime', 'Censored'])

# PCA
pca = PCA_sm(data, missing='fill-em', ncomp=5, standardize=False, normalize=False, demean=False)

# Convert to dataframe
df_pca = pd.DataFrame(pca.transformed_data, columns=data.columns)
df_pca.to_csv('csv/df_pca.csv', index=False)
pca.factors.to_csv('csv/pca_factors.csv', index=False)

In [76]:
from statsmodels.multivariate.pca import PCA as PCA_sm

# Wrapper class for PCA
class PCA_sm_wrapper:

    def __init__(self, ncomp=None, standardize=True, demean=True, normalize=True, gls=False, weights=None, method='svd', missing=None, tol=5e-08, max_iter=1000, tol_em=5e-08, max_em_iter=100, svd_full_matrices=False):
        self.ncomp = ncomp
        self.standardize = standardize
        self.demean = demean
        self.normalize = normalize
        self.gls = gls
        self.weights = weights
        self.method = method
        self.missing = missing
        self.tol = tol
        self.max_iter = max_iter
        self.tol_em = tol_em
        self.max_em_iter = max_em_iter
        self.svd_full_matrices = svd_full_matrices


    def fit(self, X, y=None):
        self.pca = PCA_sm(X, ncomp=self.ncomp, standardize=self.standardize, demean=self.demean, normalize=self.normalize, gls=self.gls, weights=self.weights, method=self.method, missing=self.missing, tol=self.tol, max_iter=self.max_iter, tol_em=self.tol_em, max_em_iter=self.max_em_iter, svd_full_matrices=self.svd_full_matrices)
        return self.pca.transformed_data


    def transform(self, X):
        self.pca = PCA_sm(X, ncomp=self.ncomp, standardize=self.standardize, demean=self.demean, normalize=self.normalize, gls=self.gls, weights=self.weights, method=self.method, missing=self.missing, tol=self.tol, max_iter=self.max_iter, tol_em=self.tol_em, max_em_iter=self.max_em_iter, svd_full_matrices=self.svd_full_matrices)
        return self.pca.transformed_data


    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

In [97]:
preprocessors = []

# For now there is only one preprocessor, but we can add more later to try different things
numerical_features = ['Age']
categorical_features = ['Gender', 'Stage', 'GeneticRisk', 'TreatmentType', 'ComorbidityIndex', 'TreatmentResponse']

# Preprocessors for missing values
numeric_inputers = [('simpleimputer', SimpleImputer(strategy='mean')), ('iterativeimputer', IterativeImputer()), ('knnimputer', KNNImputer())]

categorical_inputers = [('simpleimputer', SimpleImputer(strategy='most_frequent')), ('iterativeimputer', IterativeImputer()), ('knnimputer', KNNImputer())]


# Create multiple combinations of preprocessors
for numeric_imputer, categorical_inputer in zip(numeric_inputers, categorical_inputers):

    numeric_transformer = Pipeline(steps=[('imputer', numeric_imputer),
                                          ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[('imputer', categorical_inputer),
                                              ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])

    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numerical_features),
                                                       ('cat', categorical_transformer, categorical_features)])

    preprocessors.append(preprocessor)


# This preprocessors uses the PCA's built-in imputation and scaling
preprocessor_smpca = Pipeline(steps=[('smpca', PCA_sm_wrapper(missing='fill-em', ncomp=5))])
preprocessors.append(preprocessor_smpca)

# Supervised Learning

In [103]:
# This method uses only the labeled subset to train the model
supervised_pipelines = {}   # Dictionary of pipelines (model_name: list of pipelines)

models = [('LinearRegression', LinearRegression()), ('Ridge', Ridge()), ('KNeighborsRegressor', KNeighborsRegressor()), ('DecisionTreeRegressor', DecisionTreeRegressor()), ('RandomForestRegressor', RandomForestRegressor()), ('SVR', SVR())]

# Initialize pipeline list for each type model
for model_name, model in models:
    supervised_pipelines[model_name] = []


# Create multiple combinations of preprocessors and models
for preprocessor in preprocessors:
    for model_name, model in models:
        supervised_pipelines[model_name].append(Pipeline(steps=[('preprocessor', preprocessor),
                                                                    ('model', model)]))


# Train your model on the training data
def train_model(model_name, X_train, y_train):
    model = supervised_pipelines[model_name]
    model.fit(X_train, y_train)
    return model

def predict(model_name, X):
    model = supervised_pipelines[model_name]
    return model.predict(X)

# Train all models
def train_all_models(X_train, y_train):
    for model_name in supervised_pipelines:
        train_model(model_name, X_train, y_train)


In [104]:
supervised_pipelines

{'LinearRegression': [Pipeline(steps=[('preprocessor',
                   ColumnTransformer(transformers=[('num',
                                                    Pipeline(steps=[('imputer',
                                                                     ('simpleimputer',
                                                                      SimpleImputer())),
                                                                    ('scaler',
                                                                     StandardScaler())]),
                                                    ['Age']),
                                                   ('cat',
                                                    Pipeline(steps=[('imputer',
                                                                     ('simpleimputer',
                                                                      SimpleImputer(strategy='most_frequent'))),
                                                            

In [80]:
# Train linear regression model
model = train_model('LinearRegression')

# Predict on test set
y_pred = predict('LinearRegression', X_test)

# To csv    
df_pred = pd.DataFrame(y_pred, columns=['SurvivalTime'])
df_pred.to_csv('csv/df_pred.csv', index=False)

# Unsupervised Learning

In [23]:
features = df.drop(columns=['SurvivalTime', 'Censored'])

# Choose the number of clusters (K)
k = 5

# Apply K-Means
kmeans = KMeans(n_clusters=k)

# Visualize clusters using PCA
pca = PCA(n_components=5)

# Create a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor_smpca),
                           ('pca', pca),
                           ('clustering', kmeans)])

# Fit pipeline to data
pipeline.fit(features)


# Clustering on labeled data
# ...
"""
# Assign clusters to unlabeled data
unlabeled_features = unlabeled_df.drop(['SurvivalTime', 'Censored'], axis=1)
unlabeled_clusters = kmeans.predict(unlabeled_features)

# Merge labeled and pseudo-labeled data
unlabeled_df['Cluster'] = unlabeled_clusters
merged_df = pd.concat([labeled_df, unlabeled_df], ignore_index=True)

# Train a supervised model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Example model

X = merged_df.drop(['SurvivalTime', 'Censored'], axis=1)
y = merged_df['SurvivalTime']

# Split the merged data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a supervised model using the merged data
model = LinearRegression()  # Example model, replace with your choice
model.fit(X_train, y_train)

# Evaluate on validation set
# ...
"""

"\n# Assign clusters to unlabeled data\nunlabeled_features = unlabeled_df.drop(['SurvivalTime', 'Censored'], axis=1)\nunlabeled_clusters = kmeans.predict(unlabeled_features)\n\n# Merge labeled and pseudo-labeled data\nunlabeled_df['Cluster'] = unlabeled_clusters\nmerged_df = pd.concat([labeled_df, unlabeled_df], ignore_index=True)\n\n# Train a supervised model\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LinearRegression  # Example model\n\nX = merged_df.drop(['SurvivalTime', 'Censored'], axis=1)\ny = merged_df['SurvivalTime']\n\n# Split the merged data\nX_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Train a supervised model using the merged data\nmodel = LinearRegression()  # Example model, replace with your choice\nmodel.fit(X_train, y_train)\n\n# Evaluate on validation set\n# ...\n"

# Training and Evaluation

In [82]:
censored = df['Censored']

def cMSE(y_hat, y, c):
  err = y-y_hat
  err = (1-c)*err**2 + c*np.maximum(0,err)**2
  return np.sum(err)/err.shape[0]


#print("cMSE: %.4f" % cMSE(y_pred, y_test, censored))

def calc_fold(pipeline, X, y, train_ix, test_ix, C):
    """return classification error for train and test"""
    pipeline.fit(X[train_ix, :], y[train_ix])
    prob = reg.predict_proba(X[:, :feats])[:,1]
    squares = (prob-y)**2
    return (np.mean(squares[train_ix]), np.mean(squares[test_ix]))

cMSE: 0.3513


In [ ]:
X_labeled = df_labeled.drop(columns=['SurvivalTime', 'Censored'])

y_labeled = df_labeled['SurvivalTime']

X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)


folds = 5
kf = KFold(n_splits = folds)
lowest_val_err = 100000
errs=[]
for model_name in supervised_pipelines:  # for each set of pipelines (that use the same model)

    for pipeline in supervised_pipelines[model_name]:    # for each pipeline in the set
      
        tr_err = va_err = 0
            
        for tr_ix, va_ix in kf.split(y_train, y_train):
            r, v = calc_fold(pipeline, X_train, y_train, tr_ix, va_ix)  # (r = training error, v = validation error) for current pipeline
            tr_err += r
            va_err += v

        tr_err = tr_err/folds
        va_err = va_err/folds
        print(feats, ':', tr_err, va_err)
        errs.append((tr_err,va_err))
            
        if(va_err < lowest_val_err):
            lowest_val_err = va_err
            best_feats = feats
            
errs = np.array(errs)
fig = plt.figure(figsize=(8,8), frameon=True)
plt.plot(range(2,17), errs[:,0], '-b', linewidth=3)  # training error (blue line)
plt.plot(range(2,17), errs[:,1], '-r', linewidth=3)  # validation error(red line)
plt.show()
#plt.savefig('L05-lagreg-error.png', dpi=300, bbox_inches="tight")
plt.close()
test_error=calc_test_error(best_feats, X_train, y_train, X_test, y_test, 1e12)
create_plot(X_train, y_train, X_test, y_test, best_feats, 1e12)
print("best feats:", best_feats,":", "test_error:", test_error)

# Save Model

In [86]:
model = supervised_pipelines['LinearRegression']

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Submission Prediction

In [83]:
final_test_dataset = pd.read_csv('csv/test_data.csv').drop(columns=['id'])
final_test_dataset.head()
submission_pred = predict('LinearRegression', final_test_dataset)

In [84]:
submission_pred_df = pd.DataFrame(submission_pred, columns=['TARGET'])
submission_pred_df.insert(0, 'id', range(0, len(submission_pred_df)))
submission_pred_df.to_csv('submission_pred.csv', index=False)